### Methods: 2. Data Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Make species occurrence dataframe
lecy_occ = pd.read_csv('lecy_occ1.csv')
# Get how many columns and rows the dataframe contains
lecy_occ.shape

/Users/linli/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(80202, 50)

In [3]:
# Create subset dataframe with columns required for 
lecy_subset = lecy_occ[['genus','species','scientificName','countryCode',
                        'decimalLatitude','decimalLongitude','year']]

In [4]:
# Drop rows without Lat,Long and year values
lecy_clean = lecy_subset.dropna(subset=['decimalLongitude', 'decimalLongitude','year'])
lecy_clean.shape

(46661, 7)

In [5]:
lecy_clean.head()

,genus,species,scientificName,countryCode,decimalLatitude,decimalLongitude,year
0,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,TH,8.733881,98.234623,2021.0
1,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367390,103.828598,2021.0
2,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367460,103.828912,2021.0
3,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,SG,1.306527,103.788879,2021.0
4,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367354,103.828629,2021.0


In [6]:
# Convert the year column from float to int
lecy_clean['year'] = lecy_clean['year'].values.astype(np.int64)

<ipython-input-6-1a4b97cdb3c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lecy_clean['year'] = lecy_clean['year'].values.astype(np.int64)


In [7]:
# Get column types
lecy_clean.dtypes

genus                object
species              object
scientificName       object
countryCode          object
decimalLatitude     float64
decimalLongitude    float64
year                  int64
dtype: object

In [8]:
lecy_clean.head()

,genus,species,scientificName,countryCode,decimalLatitude,decimalLongitude,year
0,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,TH,8.733881,98.234623,2021
1,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367390,103.828598,2021
2,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367460,103.828912,2021
3,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,SG,1.306527,103.788879,2021
4,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367354,103.828629,2021


In [9]:
# Convert years to decades
lecy_clean['decade'] = (10 * (lecy_clean['year'] // 10))

<ipython-input-9-2e7a3b2ba501>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lecy_clean['decade'] = (10 * (lecy_clean['year'] // 10))


In [10]:
lecy_clean.head()

,genus,species,scientificName,countryCode,decimalLatitude,decimalLongitude,year,decade
0,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,TH,8.733881,98.234623,2021,2020
1,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367390,103.828598,2021,2020
2,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367460,103.828912,2021,2020
3,Barringtonia,Barringtonia asiatica,Barringtonia asiatica (L.) Kurz,SG,1.306527,103.788879,2021,2020
4,Couroupita,Couroupita guianensis,Couroupita guianensis Aubl.,SG,1.367354,103.828629,2021,2020


In [11]:
#lecy_clean.decade.unique()
lecy_final = lecy_clean

### Methods: 3. Map Construction

In [12]:
# Import libraries
import chart_studio.plotly as py
import plotly.graph_objs as go
fig = go.Figure()
from plotly.offline import iplot, init_notebook_mode
import plotly
import pandas as pd

In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [14]:
# Set user, API key and access token
import chart_studio
chart_studio.tools.set_credentials_file(username='lincarrieli', api_key='qOjfMIuYNEW2UO6As1q1')
#plotly.tools.set_credentials_file(username='lincarrieli', api_key='qOjfMIuYNEW2UO6As1q1')
mapbox_access_token = 'pk.eyJ1IjoibGluY2FycmllbGkiLCJhIjoiY2tvcDVpOWRuMDFtaTJ1b3d4cHZzMzlneiJ9.nb-G9SBC_2OjWi1_Ufc_xA'

In [15]:
# Get unique genus names and decade to lists
import numpy as np
genus_list = sorted(lecy_final['genus'].unique())
decade_list = sorted(lecy_final['decade'].unique())

In [16]:
# Loop over genera list and use it to filter the dataframe
data = []
for genus in genus_list:
    genus_data = dict(
            lat = lecy_final.loc[lecy_final['genus'] == genus,'decimalLatitude'],
            lon = lecy_final.loc[lecy_final['genus'] == genus,'decimalLongitude'],
            name = genus,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(genus_data)

#### Map Settings

In [17]:
# Basic setting of map
layout = dict(
    height = 1000,
    # set top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here need the token from Mapbox
        accesstoken = 'pk.eyJ1IjoibGluY2FycmllbGkiLCJhIjoiY2tvcDVpOWRuMDFtaTJ1b3d4cHZzMzlneiJ9.nb-G9SBC_2OjWi1_Ufc_xA',
        bearing = 0,
        center = dict(
            lat = 0,
            lon = 0
        ),
        pitch = 0,
        zoom = 3,
        style = 'light'
    )
)

In [18]:
# Map annotations
annotations = [dict(
              text = 'Distribution of Lecythidaceae,<br> from the 1700s to 2021', 
              font = dict(color = '#FFFFFF', size = 14), borderpad = 10,
              x = 0.01, y = 0.1, xref = 'paper', yref = 'paper', align = 'left', 
              showarrow = False, bgcolor = 'black'
              )]

# Assigning the annotations to the layout
layout['annotations'] = annotations

In [19]:
# Create drop-down menus: 1. by genera 2. by decade

i = 0
genus_Dicts = []
#add object for 'all'
genus_Dicts.append({'label': 'All genera', 'method': 'update', 'args': [{'visible': [True] * len(genus_list)}]})
while i < len(genus_list):
    temp_genusDict = {
        'label': genus_list[i],
        'method': 'update',
        'args': [{'visible': [False] * len(genus_list)}]
    }
    temp_genusDict['args'][0]['visible'][i] = True
    genus_Dicts.append(temp_genusDict)
    i += 1
    
# Loop through 2. decade list
i = 0
decade_Dicts = []
#add object for 'all'
decade_Dicts.append({'label': 'All years', 'method': 'update', 'args': [{'visible': [True] * len(decade_list)}]})
while i < len(decade_list):
    temp_decadeDict = {
        'label': decade_list[i],
        'method': 'update',
        'args': [{'visible': [False] * len(decade_list)}]
    }
    temp_decadeDict['args'][0]['visible'][i] = True
    decade_Dicts.append(temp_decadeDict)
    i += 1

In [20]:
updatemenus = list([
    dict(
        buttons = genus_Dicts,
        direction = 'up',
        x = 0.99,
        xanchor = 'left',
        y = 0.1,
        yanchor = 'bottom',
    
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),

    dict(
        buttons = decade_Dicts,
        direction = 'down',
        x = 0.01,
        y = 0.99,
        yanchor = 'bottom',
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)     
    )

])
layout['updatemenus'] = updatemenus
layout['title'] = 'Global Distribution of Lecythidaceae'

In [21]:
figure = dict(data = data, layout = layout)
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=figure)
])

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
